## Deploy Parte 2
<br>
    Após realizada a primeira etapa, agora vamos realizar o carregamento do arquivo, rodar os dados com a primeira base para garantir que o resultado será o mesmo e após disso, aplicar o modelo para os novos dados. Ao fim o modelo exporta um arquivo xlsx para o cliente com as previsões das vendas.

In [2]:
# Importando o load do joblib
from joblib import load

In [3]:
# Carregando o modelo que havíamos exportado
reg = load('regressor.joblib')

In [4]:
# Verificando o coef_
reg.coef_

array([-182.99973759,   31.00151165])

In [5]:
# E então o intercept_
reg.intercept_

4253.997131672601

**Utilizando o modelo para os mesmos dados**

In [6]:
# Importando a base
import pandas as pd
base = pd.read_excel('dadosVenda.xlsx')

In [7]:
# Fazendo os tratamentos
# Verificando se existem valores vazios
base.isnull().sum()

# tratando esses valores
base.loc[base.Desconto.isnull(), 'Desconto'] = 0

In [8]:
# Separando em treino e teste
X = base[['PrecoOriginal', 'Desconto']]
y = base.VendaQtd

In [9]:
# Fazendo a previsão para os dados de teste
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)

y_pred = reg.predict(X_test)

In [10]:
# Avaliando os erros
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

print(mean_absolute_error(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))

0.25078643242467186
0.0854790321083994


**Utilizando o modelo nos novos dados**

In [11]:
# Importando a base
basenova = pd.read_excel('dadosVenda_producao.xlsx')

In [12]:
# Visualizando a base
basenova.head()

,IDRegistro,PrecoOriginal,Desconto,VendaQtd
0,1605,13,2.86,NaN
1,1606,21,2.52,NaN
2,1607,5,1.95,NaN
3,1608,15,0.75,NaN
4,1609,11,0.99,NaN


**Realizando todos os tratamentos necessários para essa base**

In [13]:
# Verificando valores vazios
basenova.isnull().sum()

IDRegistro        0
PrecoOriginal     0
Desconto          1
VendaQtd         30
dtype: int64

In [14]:
# Tratando os valores vazios do Desconto
basenova.loc[basenova.Desconto.isnull(), 'Desconto'] = 0

In [15]:
# Separando X e y
X = basenova[['PrecoOriginal', 'Desconto']]

**Usando essas dados no modelo para fazer a previsão**

In [16]:
# Fazendo essa previsão
y_pred = reg.predict(X)

In [17]:
# Podemos deixar com 1 única casa decimal
import numpy
numpy.around(y_pred,1)

array([1963.7,  489.1, 3399.5, 1532.3, 2271.7,  262.1, 2008. , 2517. ,
        941.9, 1643.9,  593.3, 2809.8,  515.2, 1583.4,  137.7,  818.2,
       3223. , 2473.6, 2529.4,  777. , 1770.1, 2080.3, 3217.4, 3384. ,
       2979.5, 1907.2, 2504.6, 1597.4, 1405.4, 3710.6])

In [18]:
# Salvando a previsão em uma variável
y_pred = numpy.around(y_pred,1)

In [19]:
# Podemos substituir essa previsão na base inicial
basenova['VendaQtd'] = y_pred

In [20]:
# Visualizando novamente a base
basenova.head()

,IDRegistro,PrecoOriginal,Desconto,VendaQtd
0,1605,13,2.86,1963.7
1,1606,21,2.52,489.1
2,1607,5,1.95,3399.5
3,1608,15,0.75,1532.3
4,1609,11,0.99,2271.7


**Validando o modelo**

In [21]:
basenova['Valida'] = basenova.PrecoOriginal*reg.coef_[0]+basenova.Desconto+reg.coef_[1]+reg.intercept_

In [22]:
basenova.head()

,IDRegistro,PrecoOriginal,Desconto,VendaQtd,Valida
0,1605,13,2.86,1963.7,1908.862055
1,1606,21,2.52,489.1,444.524154
2,1607,5,1.95,3399.5,3371.949955
3,1608,15,0.75,1532.3,1540.752580
4,1609,11,0.99,2271.7,2272.991530


**Salvando esse resultado em um Excel**

In [23]:
# Exportando para o Excel
basenova.to_excel('Resultado.xlsx')